In [17]:
import opendatasets as od
import pandas as pd
import json
import cv2
import numpy as np
import tensorflow as tf
from tensorflow.keras import layers, models

In [2]:
od.download( 
    "https://www.kaggle.com/datasets/risangbaskoro/wlasl-processed") 

Please provide your Kaggle credentials to download this dataset. Learn more: http://bit.ly/kaggle-creds
Your Kaggle username: raflyrangghaniputra
Your Kaggle Key: ········
Dataset URL: https://www.kaggle.com/datasets/risangbaskoro/wlasl-processed


100%|█████████████████████████████████████████████████████████████████████████████| 4.82G/4.82G [12:08<00:00, 7.11MB/s]


In [16]:
with open('./wlasl-processed/WLASL_v0.3.json') as f:
    data = json.load(f)
    
print(type(data))  # Ini akan menunjukkan apakah data adalah list atau dict

gloss_set = set()
for item in data:
    if 'gloss' in item:
        gloss_set.add(item['gloss'])
        
gloss_to_idx = {gloss: idx for idx, gloss in enumerate(gloss_set)}
num_classes = len(gloss_set)
print(f'Jumlah kelas: {num_classes}')

<class 'list'>
Jumlah kelas: 2000


In [29]:
def extract_frames(video_path, num_frames=30, frame_size=(150, 150)):
    cap = cv2.VideoCapture(video_path)
    frames = []
    
    while cap.isOpened() and len(frames) < num_frames:
        ret, frame = cap.read()
        if not ret:
            break
        frame = cv2.resize(frame, frame_size)
        frames.append(frame)
    
    # Jika jumlah frame kurang dari `num_frames`, duplikasi frame terakhir
    while len(frames) < num_frames:
        frames.append(frames[-1])
    
    cap.release()
    return np.array(frames)

In [51]:
def preprocess(video_path, label):
    # Konversi EagerTensor ke string path
    video_path = video_path.numpy().decode("utf-8")
    # Baca video dan ekstrak frame
    frames = extract_frames(video_path)
    return frames, label

def create_dataset(video_paths, labels):
    video_paths = tf.constant(video_paths)
    labels = tf.constant(labels)

    dataset = tf.data.Dataset.from_tensor_slices((video_paths, labels))
    dataset = dataset.map(lambda video_path, label: tf.py_function(
        preprocess, inp=[video_path, label], Tout=(tf.float32, tf.int32)
    ), num_parallel_calls=tf.data.AUTOTUNE)
    # Tambahkan batching dan shuffle
    dataset = dataset.batch(8).shuffle(buffer_size=100).prefetch(tf.data.AUTOTUNE)
    return dataset

In [52]:
from tensorflow.keras import layers, models

def create_model(input_shape, num_classes):
    model = models.Sequential()
    model.add(layers.Conv3D(32, (3, 3, 3), activation='relu', input_shape=input_shape))
    model.add(layers.MaxPooling3D((2, 2, 2)))
    model.add(layers.Conv3D(64, (3, 3, 3), activation='relu'))
    model.add(layers.MaxPooling3D((2, 2, 2)))
    model.add(layers.Flatten())
    model.add(layers.Dense(128, activation='relu'))
    model.add(layers.Dense(num_classes, activation='softmax'))
    return model

# Contoh penggunaan
num_frames = 30
input_shape = (num_frames, 150, 150, 3)  # Sesuaikan dengan output extract_frames
model = create_model(input_shape, num_classes)
model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'], run_eagerly=True)

In [37]:
from sklearn.model_selection import train_test_split
import os

# Mendapatkan path video dan gloss dari JSON
video_paths = []
gloss_labels = []

video_dir = './wlasl-processed/videos'

for item in data:
    gloss = item['gloss']
    for instance in item['instances']:
        video_id = instance['video_id']
        video_path = os.path.join(video_dir, f'{video_id}.mp4')
        if os.path.exists(video_path):  # Pastikan file video ada
            video_paths.append(video_path)
            gloss_labels.append(gloss)


In [53]:
# Split dataset menjadi train dan validation
train_paths, val_paths, train_labels, val_labels = train_test_split(
    video_paths, gloss_labels, test_size=0.2, stratify=gloss_labels
)

# Buat dataset TensorFlow
train_dataset = create_dataset(train_paths, train_labels)
val_dataset = create_dataset(val_paths, val_labels)


In [54]:
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint

callbacks = [
    EarlyStopping(monitor='val_loss', patience=3, restore_best_weights=True),
    ModelCheckpoint(filepath='best_model.h5', monitor='val_loss', save_best_only=True)
]

In [55]:
epochs = 20
history = model.fit(
    train_dataset,
    validation_data=val_dataset,
    epochs=epochs,
    callbacks=callbacks
)

Epoch 1/20


InvalidArgumentError: {{function_node __wrapped__IteratorGetNext_output_types_2_device_/job:localhost/replica:0/task:0/device:CPU:0}} ValueError: Tensor conversion requested dtype int32 for Tensor with dtype string: <tf.Tensor: shape=(), dtype=string, numpy=b'jewelry'>
Traceback (most recent call last):

  File "C:\Users\rafly\AppData\Local\Programs\Python\Python311\Lib\site-packages\tensorflow\python\ops\script_ops.py", line 268, in __call__
    return func(device, token, args)
           ^^^^^^^^^^^^^^^^^^^^^^^^^

  File "C:\Users\rafly\AppData\Local\Programs\Python\Python311\Lib\site-packages\tensorflow\python\ops\script_ops.py", line 146, in __call__
    outputs = self._call(device, args)
              ^^^^^^^^^^^^^^^^^^^^^^^^

  File "C:\Users\rafly\AppData\Local\Programs\Python\Python311\Lib\site-packages\tensorflow\python\ops\script_ops.py", line 162, in _call
    outputs = [
              ^

  File "C:\Users\rafly\AppData\Local\Programs\Python\Python311\Lib\site-packages\tensor

In [48]:
print(f"Total training samples: {len(train_paths)}, Total validation samples: {len(val_paths)}")
# Coba cetak beberapa contoh untuk memastikan data benar
for video, label in create_dataset(train_paths[:2], train_labels[:2]).take(1):
    print(video.shape, label)  # Video shape harus sesuai dengan input model

Total training samples: 9584, Total validation samples: 2396


UnknownError: {{function_node __wrapped__IteratorGetNext_output_types_2_device_/job:localhost/replica:0/task:0/device:CPU:0}} AttributeError: 'tensorflow.python.framework.ops.EagerTensor' object has no attribute 'decode'
Traceback (most recent call last):

  File "C:\Users\rafly\AppData\Local\Programs\Python\Python311\Lib\site-packages\tensorflow\python\ops\script_ops.py", line 268, in __call__
    return func(device, token, args)
           ^^^^^^^^^^^^^^^^^^^^^^^^^

  File "C:\Users\rafly\AppData\Local\Programs\Python\Python311\Lib\site-packages\tensorflow\python\ops\script_ops.py", line 146, in __call__
    outputs = self._call(device, args)
              ^^^^^^^^^^^^^^^^^^^^^^^^

  File "C:\Users\rafly\AppData\Local\Programs\Python\Python311\Lib\site-packages\tensorflow\python\ops\script_ops.py", line 153, in _call
    ret = self._func(*args)
          ^^^^^^^^^^^^^^^^^

  File "C:\Users\rafly\AppData\Local\Programs\Python\Python311\Lib\site-packages\tensorflow\python\autograph\imp

In [49]:
for video, label in create_dataset(train_paths[:2], train_labels[:2]).take(1):
    print("Video shape:", video.shape)  # Harus (batch_size, num_frames, height, width, channels)
    print("Label shape:", label.shape)  # Harus (batch_size,)

UnknownError: {{function_node __wrapped__IteratorGetNext_output_types_2_device_/job:localhost/replica:0/task:0/device:CPU:0}} AttributeError: 'tensorflow.python.framework.ops.EagerTensor' object has no attribute 'decode'
Traceback (most recent call last):

  File "C:\Users\rafly\AppData\Local\Programs\Python\Python311\Lib\site-packages\tensorflow\python\ops\script_ops.py", line 268, in __call__
    return func(device, token, args)
           ^^^^^^^^^^^^^^^^^^^^^^^^^

  File "C:\Users\rafly\AppData\Local\Programs\Python\Python311\Lib\site-packages\tensorflow\python\ops\script_ops.py", line 146, in __call__
    outputs = self._call(device, args)
              ^^^^^^^^^^^^^^^^^^^^^^^^

  File "C:\Users\rafly\AppData\Local\Programs\Python\Python311\Lib\site-packages\tensorflow\python\ops\script_ops.py", line 153, in _call
    ret = self._func(*args)
          ^^^^^^^^^^^^^^^^^

  File "C:\Users\rafly\AppData\Local\Programs\Python\Python311\Lib\site-packages\tensorflow\python\autograph\imp